In [ ]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# Define the transformations including flattening and normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the image to 1D array
])

In [ ]:
# Load the MNIST training and test datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Verify shapes
data_iter = iter(train_loader)
images, labels = next(data_iter)  # Use next() function
print(f'Images batch shape: {images.shape}')
print(f'Labels batch shape: {labels.shape}')

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 16384361.78it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 449131.45it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4114559.44it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4005577.96it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Images batch shape: torch.Size([64, 784])
Labels batch shape: torch.Size([64])


Defining Nearest Neighbor Classifier

In [ ]:
import numpy as np

class NearestNeighbor:
    def __init__(self, k=1):
        self.k = k

    def train(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        num_test = X.shape[0]
        Y_pred = np.zeros(num_test, dtype=self.y_train.dtype)

        for i in range(num_test):
            distances = np.sum(np.abs(self.X_train - X[i, :]), axis=1)
            min_indices = np.argsort(distances)[:self.k]
            closest_y = self.y_train[min_indices]
            Y_pred[i] = np.bincount(closest_y).argmax()

        return Y_pred

Hyperparameter Tuning with 5-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def cross_validate(k_values, X_train, y_train):
    kf = KFold(n_splits=5)
    accuracies = {k: [] for k in k_values}

    for train_index, val_index in kf.split(X_train):
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, y_val = y_train[train_index], y_train[val_index]

        for k in k_values:
            nn = NearestNeighbor(k)
            nn.train(X_tr, y_tr)
            y_pred = nn.predict(X_val)
            accuracy = accuracy_score(y_val, y_pred)
            accuracies[k].append(accuracy)

    avg_accuracies = {k: np.mean(v) for k, v in accuracies.items()}
    return avg_accuracies

k_values = [1, 2, 5, 10, 20]
train_data = train_dataset.data.numpy().reshape(-1, 28*28)
train_labels = train_dataset.targets.numpy()
accuracies = cross_validate(k_values, train_data, train_labels)

# Plotting accuracies
import matplotlib.pyplot as plt

plt.figure()
plt.plot(k_values, [accuracies[k] for k in k_values])
plt.xlabel('k')
plt.ylabel('Validation Accuracy')
plt.title('5-Fold Cross-Validation Accuracy for Different k')
plt.show()

best_k = max(accuracies, key=accuracies.get)
print(f'Best value of k: {best_k}')

KeyboardInterrupt: 

Evaluating the Model

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

nn = NearestNeighbor(best_k)
nn.train(train_data, train_labels)

test_data = test_dataset.data.numpy().reshape(-1, 28*28)
test_labels = test_dataset.targets.numpy()

y_pred = nn.predict(test_data)

accuracy = accuracy_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred, average='macro')
recall = recall_score(test_labels, y_pred, average='macro')
f1 = f1_score(test_labels, y_pred, average='macro')
conf_matrix = confusion_matrix(test_labels, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print('Confusion Matrix:')
print(conf_matrix)